In [ ]:
%run ../notebook_setup.py

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import starry
import pymc3 as pm
import pymc3_ext as pmx

## Generate

In [ ]:
map = starry.Map(oblate=True, lazy=False)

In [ ]:
truths = {
    "f": 0.3,
    "omega": 0.5,
    "inc": 60,
    "obl": 30,
    "b0": 0.5,
    "ro": 0.1
}

ferr = 3e-4

In [ ]:
map.f = truths["f"]
map.omega = truths["omega"]
map.inc = truths["inc"]
map.obl = truths["obl"]

In [ ]:
xo = np.linspace(-1, 1, 300)
yo = truths["b0"] * np.ones_like(xo)
ro = truths["ro"]

In [ ]:
fig, ax = plt.subplots(1)
ax.plot(xo, yo, "k-")
ax.plot(xo, yo - ro, "k--")
ax.plot(xo, yo + ro, "k--")
map.show(ax=ax)

In [ ]:
flux0 = map.flux(xo=xo, yo=yo, ro=ro)
flux = flux0 + ferr * np.random.randn(len(flux0))

In [ ]:
plt.plot(flux0);
plt.plot(flux, "k.", ms=3, alpha=0.75);

## Infer

In [ ]:
with pm.Model() as model:
    f = pm.Uniform("f", lower=0.0, upper=0.9, testval=truths["f"])
    omega = pm.Uniform("omega", lower=0.0, upper=0.9, testval=truths["omega"])
    inc = pm.Uniform("inc", lower=0.0, upper=90.0, testval=truths["inc"])
    obl = pm.Uniform("obl", lower=-90, upper=90.0, testval=truths["obl"])
    b0 = pm.Uniform("b0", lower=-1.0, upper=1.0, testval=truths["b0"])
    ro = pm.Uniform("ro", lower=1e-5, upper=0.5, testval=truths["ro"])
    map = starry.Map(oblate=True)
    map.f = f
    map.omega = omega
    map.inc = inc
    map.obl = obl
    yo = b0 * np.ones_like(xo)
    flux_model = map.flux(xo=xo, yo=yo, ro=ro)
    pm.Normal("obs", mu=flux_model, sd=ferr, observed=flux)

In [ ]:
np.random.seed(42)
with model:
    trace = pmx.sample(
        tune=1000,
        draws=1000,
        chains=2,
        cores=1,
        target_accept=0.9,
        return_inferencedata=True,
    )

In [ ]:
pm.summary(trace)

In [ ]:
pm.plot_trace(trace);